## Ens

In [65]:
! sh ens.sh

+ OUT_EXT=.out.tmp
++ ls 1=89.5.txt 2=90.34.txt 3=89.68.txt 4=89.17.txt
+ TXTS='1=89.5.txt
2=90.34.txt
3=89.68.txt
4=89.17.txt'
+ for f in $TXTS
+ dos2unix 1=89.5.txt
dos2unix: converting file 1=89.5.txt to Unix format...
+ OUT=1=89.5.txt.out.tmp
+ echo 1=89.5.txt
+ echo -e '\n'
+ awk -F '	' '{print $2}' 1=89.5.txt
+ for f in $TXTS
+ dos2unix 2=90.34.txt
dos2unix: converting file 2=90.34.txt to Unix format...
+ OUT=2=90.34.txt.out.tmp
+ echo 2=90.34.txt
+ echo -e '\n'
+ awk -F '	' '{print $2}' 2=90.34.txt
+ for f in $TXTS
+ dos2unix 3=89.68.txt
dos2unix: converting file 3=89.68.txt to Unix format...
+ OUT=3=89.68.txt.out.tmp
+ echo 3=89.68.txt
+ echo -e '\n'
+ awk -F '	' '{print $2}' 3=89.68.txt
+ for f in $TXTS
+ dos2unix 4=89.17.txt
dos2unix: converting file 4=89.17.txt to Unix format...
+ OUT=4=89.17.txt.out.tmp
+ echo 4=89.17.txt
+ echo -e '\n'
+ awk -F '	' '{print $2}' 4=89.17.txt
++ sort -n
++ ls 1=89.5.txt.out.tmp 2=90.34.txt.out.tmp 3=89.68.txt.out.tmp 4=89.17.txt.out.tmp
+ OUT

## Read ens_out

In [66]:
ens_out_file = "ens_out"
voting_file = f"voting.txt"
weighted_voting_file = f"weighted_voting.txt"

In [67]:
with open(ens_out_file) as f:
    lines = f.readlines()
    
accs = [float(item.split("=")[1].replace(".txt", "")) for item in lines[0].split("\t")]
print(accs)

lines = lines[3:]

[89.5, 90.34, 89.68, 89.17]


In [68]:
from collections import Counter

## Voting

In [69]:
predictions = []
for line in lines:
    if line.strip() == "":
        predictions.append("\n")
        continue
    
    line_predictions = line.strip().split("\t")
    line_prediction = Counter(line_predictions).most_common()[0][0]
    predictions.append(line_prediction)

In [70]:
with open(voting_file, "w") as f:
    for prediction in predictions:
        f.write(f"{prediction}")
        if prediction != "\n":
            f.write("\n")

## Weighted-voting

In [71]:
norm_accs = [acc / sum(accs) for acc in accs]
print(norm_accs)

weighted_predictions = []
for line in lines:
    if line.strip() == "":
        weighted_predictions.append("\n")
        continue
    
    line_predictions = line.strip().split("\t")
    line_predictions_dict = {}
    for i in range(len(line_predictions)):
        line_prediction = line_predictions[i]
        norm_acc = norm_accs[i]
        
        if line_prediction not in line_predictions_dict.keys():
            line_predictions_dict[line_prediction] = norm_acc
        else:
            line_predictions_dict[line_prediction] += norm_acc
            
    line_prediction = sorted(list(line_predictions_dict.items()), key=lambda item: item[1], reverse=True)[0][0]
    weighted_predictions.append(line_prediction)

[0.24951908333101006, 0.2518609384147871, 0.2500209094203909, 0.24859906883381194]


In [72]:
with open(weighted_voting_file, "w") as f:
    for weighted_prediction in weighted_predictions:
        f.write(f"{weighted_prediction}")
        if weighted_prediction != "\n":
            f.write("\n")

## Cmp

In [73]:
! paste "Answer_Same.tsv.cleaned" voting.txt > voting_srcs_trgs_outs.txt
! paste "Answer_Same.tsv.cleaned" weighted_voting.txt > weighted_voting_srcs_trgs_outs.txt

! awk '{printf "%s\t%s\n", $2, $3}' voting_srcs_trgs_outs.txt  > voting_trgs_outs.txt
! awk '{printf "%s\t%s\n", $2, $3}' weighted_voting_srcs_trgs_outs.txt  > weighted_voting_trgs_outs.txt

! rm voting_srcs_trgs_outs.txt
! rm weighted_voting_srcs_trgs_outs.txt

In [74]:
def get_acc(trgs, outs):
    assert len(trgs) == len(outs)
    
    right_count = 0
    count = 0
    for trg, out in zip(trgs, outs):
        if trg == out:
            right_count += 1
        count += 1
        
    return right_count / count

In [75]:
def get_pairs(file):
    with open(file) as f:
        lines = f.readlines()
    
    trgs = []
    outs = []
    for line in lines:
        if line.strip() == "":
            continue
        
        trg, out = line.strip().split("\t")
        trgs.append(trg)
        outs.append(out)
    
    return trgs, outs

In [76]:
print(get_acc(*get_pairs("voting_trgs_outs.txt")))

0.8991058886485763


In [77]:
print(get_acc(*get_pairs("weighted_voting_trgs_outs.txt")))

0.9032754353497291
